In [1]:
import torch
import torch.nn as nn
from torch.nn.utils import prune

In [2]:
print(f'PyTorch version : {torch.__version__}\n'
      f'CUDA version : {torch.cuda_version}')

PyTorch version : 1.13.1
CUDA version : 11.7


In [5]:
fc = nn.Linear(100, 200)

params = dict(fc.named_parameters())
params.keys()

dict_keys(['weight', 'bias'])

In [6]:
buffers = dict(fc.named_buffers())
buffers.keys()

dict_keys([])

In [7]:
prune.l1_unstructured(fc, 'weight', 0.5)

Linear(in_features=100, out_features=200, bias=True)

In [8]:
params = dict(fc.named_parameters())
params.keys()

dict_keys(['bias', 'weight_orig'])

In [9]:
buffers = dict(fc.named_buffers())
buffers.keys()

dict_keys(['weight_mask'])

In [12]:
fc.weight

tensor([[ 0.0000,  0.0000, -0.0989,  ...,  0.0000, -0.0883, -0.0000],
        [-0.0798,  0.0000, -0.0627,  ...,  0.0577,  0.0769, -0.0962],
        [-0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0555],
        ...,
        [ 0.0000,  0.0000,  0.0773,  ...,  0.0000, -0.0000, -0.0576],
        [-0.0786,  0.0535, -0.0599,  ...,  0.0660,  0.0000, -0.0654],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.0000, -0.0620, -0.0000]],
       grad_fn=<MulBackward0>)

In [11]:
fc._forward_pre_hooks

OrderedDict([(0, <torch.nn.utils.prune.L1Unstructured at 0x202483f4c40>)])

In [15]:
random_tensor = torch.randn(100, 200)

l1_unstructured = prune.L1Unstructured(0.5)
pruned_tensor = l1_unstructured.prune(random_tensor)

pruned_tensor

tensor([[ 1.1987,  1.2184, -0.0000,  ...,  0.7120, -0.0000, -0.0000],
        [ 0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.8938,  2.0662],
        [-1.0300,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
        ...,
        [-1.0800, -0.8545,  0.0000,  ..., -0.0000, -1.2614,  0.8715],
        [-0.7051,  0.0000,  1.0982,  ...,  1.4157, -1.0476,  0.0000],
        [ 0.0000,  0.7177,  0.0000,  ..., -0.7013, -0.0000,  0.0000]])

In [17]:
class L1UnstructuredPartitioned(prune.L1Unstructured):

    def __init__(self, amount, npartitions):
        super(L1UnstructuredPartitioned, self).__init__(amount)
        self.npartitions = npartitions


    def compute_mask(self, t, default_mask):
        t_partitions = torch.split(t, t.shape[0] // self.npartitions, dim=0)
        default_mask_partitions = torch.split(
            default_mask, default_mask.shape[0] // self.npartitions, dim=0
        )
        mask_partitions = [
            super(L1UnstructuredPartitioned, self).compute_mask(t_p, dm_p)
            for t_p, dm_p in zip(t_partitions, default_mask_partitions)
        ]
        return torch.concat(mask_partitions, dim=0)
    

    @classmethod
    def apply(cls, module, name, amount, npartitions, importance_scores=None):
        return super(prune.L1Unstructured, cls).apply(
            module, name, amount, npartitions, importance_scores=importance_scores
        )
    


def l1_unstructured_partitioned(
        module, name, amount, npartitions, importance_scores=None
    ):
    L1UnstructuredPartitioned.apply(
        module,
        name,
        amount,
        npartitions,
        importance_scores=importance_scores
    )
    return module

In [18]:
fc = nn.Linear(100, 300)

l1_unstructured_partitioned(fc, 'weight', 0.5, 3)

Linear(in_features=100, out_features=300, bias=True)

In [21]:
partitioned_weights = torch.split(fc.weight, int(fc.weight.shape[0] / 3))

for idx, partitioned_weight in enumerate(partitioned_weights):
    num_units_in_partition = partitioned_weight.nelement()
    zeros = (partitioned_weight == 0.).sum()
    print(f"Partition {idx} sparsity : {zeros / num_units_in_partition:.2f}")

Partition 0 sparsity : 0.50
Partition 1 sparsity : 0.50
Partition 2 sparsity : 0.50
